# Решающие деревья, решающие леса и градиентный бустинг.

## 1. Решающее дерево vs решающий лес. 

Будем сравнивать эти алгоритмы на примере задачи регрессии: задачи предсказания стоимости домов в Бостоне.

In [ ]:
from sklearn.datasets import load_boston

data = load_boston()
print(data.DESCR)

In [ ]:
X_full = data.data
y_full = data.target

In [ ]:
X_full.shape

In [ ]:
y_full[:10]

Будем оценивать качество алгоритмов по кросс-валидации.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score

model_lin = LinearRegression()

score = cross_val_score(model_lin, X_full, y_full, cv=3, scoring='r2').mean()

print(score)

### Задание 1.

Посмотрите на качество решающего дерева на кросс-валидации. Запустите ячейку несколько раз и посмотрите, меняется ли качество.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

#your code here

### Задание 2.

Посмотрите на качество случайного леса на кросс-валидации. Запустите ячейку несколько раз и посмотрите, меняется ли качество.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

#your code here

Подберём параметры у случайного леса:
    
    * n_estimators - число деревьев в лесе
    * max_depth - максимальная глубина деревьев
    * max_features - максимальное число признаков для построения каждого дерева
    * min_samples_split - минимальное число объектов, необходимое для разбиения внутренней вершины
    
и другие, см. https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

Самый важный параметр - число деревьев в лесе. Начнём с него.

### Задание 3. 

Постройте лес из 10, 100, 500 и 1000 деревьев и посмотрите на r2 на кросс-валидации.

Примените цикл по количеству деревьев.

In [ ]:
for n in [10, 100, 500, 1000, 10000]:
    print('n_estimators:', n)
    
    model_rf = #your code here

    score = #your code here

    print('r2:', score)

Мы видим, что чем больше деревьев в лесе, тем дольше он обучается. Однако качество алгоритма при 1000 и 10000 деревьев различается незначительно. Добавим в предыдущий цикл время работы алгоритма.

In [ ]:
from time import time

for n in [10, 100, 500, 1000, 10000]:
    print('n_estimators:', n)
    model_rf = RandomForestRegressor(n_estimators=n)

    score = cross_val_score(model_rf, X_full, y_full, cv=3, scoring='r2').mean()

    t_end = time()
    print('time:', t_end-t_start)
    print('r2:', score)
    print()

Решающий лес устроим таким образом, что чем больше деревьев в лесе, тем лучше качество предсказания. Однако, начиная с некоторого количества деревьев, качество увеличивается незначительно. А время, затрачиваемое на обучение алгоритма, увеличивается с увеличением числа деревьев. Построим график качества алгоритма в зависимости от количества деревьев.

In [ ]:
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import cross_val_score
from time import time

r2_list = []
time_list = []

n_list = np.arange(10,2000,100)

for n in tqdm(n_list):
    model_rf = RandomForestRegressor(n_estimators=n)

    t_start = time()

    score = cross_val_score(model_rf, X_full, y_full, cv=3, scoring='r2').mean()

    t_end = time()

    r2_list.append(score)
    time_list.append(t_end-t_start)

In [ ]:
from matplotlib import pylab as plt
%pylab inline

plot(n_list, r2_list, '-o')
show()

In [ ]:
plot(n_list, time_list, '-o')
show()

Будем использовать лес с 600 деревьями.

Далее будем подбирать параметры max_depth, max_features и min_samples_split с помощью функции GridSearchCV.

In [ ]:
from sklearn.model_selection import GridSearchCV

model_rf = RandomForestRegressor(n_estimators=600, n_jobs=-1, random_state=123)

params = {'max_features': [None, 'log2', 'sqrt'], 
          'max_depth': [2, 4, 6, 8, 10, 20, 50]}

gs = GridSearchCV(model_rf,
                  params,
                  cv=3,
                  scoring='r2',
                  n_jobs=-1)
gs.fit(X_full, y_full)

In [ ]:
gs.best_estimator_, gs.best_score_

### Задание 4. 

Попробуйте одновременно подбрать max_features, max_depth и min_samples_leaf. Последний параметр ищите по списку [1, 5, 10, 15, 20].

Улучшилось ли качество алгоритма при одновременном подборе трех параметров?

In [ ]:
from sklearn.model_selection import GridSearchCV

#your code here

In [ ]:
gs.best_estimator_, gs.best_score_

## 2. Решающий лес vs градиентный бустинг.

Теперь будем решать задачу классификации ([Kaggle: Predicting a Biological Response](https://www.kaggle.com/c/bioresponse)).

In [ ]:
import pandas as pd

data = pd.read_csv('train.csv')
print(data.shape)
X = data.iloc[:, 1:].values
y = data.iloc[:, 0].values
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.8, random_state=241)

In [ ]:
len(y[y==0]), len(y[y==1])

### Задание 5.

Посмотрите на качество решающего леса с 10 деревьями на кросс-валидации (здесь метрика - accuracy, её можно не передавать в функцию cross_val_score, так как она там стоит по умолчанию для задачи классификации).

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

#your code here

### Задание 6.

Посмотрите на качество градиентного бустинга с 10 деревьями на кросс-валидации.

In [ ]:
#your code here

Нарисуем графики качества леса на тренировочной и тестовой выборках в зависимости от числа деревьев. Также нарисуем график времени работы алгоритма от числа деревьев.

In [ ]:
from sklearn.metrics import accuracy_score
from time import time

acc_train = []
acc_test = []
time_list = []

n_list = np.arange(10,250,20)

for n in tqdm(n_list):
    model_rf = RandomForestClassifier(n_estimators=n)

    t_start = time()
    model_rf.fit(Xtrain, ytrain)

    pred_train = model_rf.predict(Xtrain)
    acc_train.append(accuracy_score(ytrain, pred_train))
    
    pred_test = model_rf.predict(Xtest)
    acc_test.append(accuracy_score(ytest, pred_test))

    t_end = time()

    time_list.append(t_end-t_start)

In [ ]:
figure(figsize=(12, 4))
subplot(1, 3, 1)
plot(n_list, acc_train, label='RF_train')
plt.legend()
plt.title('Train')
plt.subplot(1, 3, 2)
plot(n_list, acc_test, label='RF_test')
plt.legend()
plt.title('Test')
plt.subplot(1, 3, 3)
plot(n_list, time_list, label='RF_time')
plt.legend()
plt.title('Time')

### Задание 7.

Теперь посмотрим, как меняется качество бустинга при увеличении числа деревьев. 

Постройте аналогичные графики, но для градиентного бустинга.

In [ ]:
#your code here

Мы видим, что у бустинга ошибка на Train всё время уменьшается, а ошибка на Test начинает расти, начиная с некоторого количества деревьев. То есть, начиная с этого числа деревьев алгоритм переобучается. Значит, нам надо остановиться на 
некотором числе деревьев, которому на графике ошибки на тесте соответствует наименьшая ошибка.

### Задание 8. 

После выбора числа деревьев остальные параметры у бустинга можно подбирать с помощью GridSearchCV.
Подберите max_features, max_depth и min_samples_leaf для бустинга с помощью GridSearchCV. 

In [ ]:
#your code here

In [ ]:
gs.best_estimator_, gs.best_score_

### Вывод.

И решающий лес, и градиентный бустинг - это мощные ансамблевые алгоритмы, которые при должной настройке параметров показывают хорошее качество работы. Но надо помнить, что градиентный бустинг имеет склонность к переобучению с увеличением числа деревьев, и аккуратно подбирать число деревьев для алгоритма. 